In [1]:
import pandas as pd
from pyspark.sql.functions import *

In [2]:
from pyspark.sql import SparkSession

# Create a spark session
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true") 
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/10/03 12:16:27 WARN Utils: Your hostname, LAPTOP-UQ7BDL4T resolves to a loopback address: 127.0.1.1; using 172.24.128.113 instead (on interface eth0)
22/10/03 12:16:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/03 12:16:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
transaction_df = spark.read.parquet('../data/curated/transactions_detail.parquet')

In [4]:
transaction_count_df = transaction_df.groupBy('merchant_name', 'postcode').count()
transaction_count_df = transaction_count_df.withColumnRenamed("count","transaction_count")

In [5]:
from pyspark.sql import Window
import pyspark.sql.functions as f

w = Window.partitionBy('merchant_name')
max_count_df = transaction_count_df.withColumn('max_count', f.max('transaction_count').over(w))\
                .where(f.col('transaction_count') == f.col('max_count'))\
                .drop('max_count')\
                .dropDuplicates(['merchant_name'])


In [6]:
merchant_detail = spark.read.parquet('../data/curated/merchant_detail.parquet')

In [7]:
merchant_detail = merchant_detail.join(max_count_df.drop(col("transaction_count")),['merchant_name'],how='inner')

In [13]:
merchant_detail = merchant_detail.toPandas()
merchant_detail['postcode'] = merchant_detail['postcode'].astype(int)

In [10]:
income_df = pd.read_csv("../data/income/2021_income.csv")
income_df['POA_CODE_2021'] = income_df['POA_CODE_2021'].astype(int)

In [14]:
merged_df = merchant_detail \
    .merge(income_df[['POA_CODE_2021', 'Median_tot_prsnl_inc_weekly']], left_on='postcode', right_on='POA_CODE_2021') \
    .drop('POA_CODE_2021', axis=1)


In [15]:
merged_df

,merchant_name,merchant_abn,type,take_rate,total_income,total_transactions,mean_transaction_amount,monthly_profit,postcode,Median_tot_prsnl_inc_weekly
0,A Associates,14955471775,"opticians, optical goods, and eyeglasses",4.95,24797.58,71,349.0,68.0,4620,476
1,Sollicitudin Orci Inc.,29839336901,shoe shops,4.54,17246.95,72,240.0,44.0,4620,476
2,Ante Vivamus Industries,57540672941,bicycle shops - sales and service,4.71,490063.47,609,805.0,1282.0,4620,476
3,A Felis Ullamcorper Corporation,36800720158,"florists supplies, nursery stock, and flowers",6.04,509614.30,1155,441.0,1710.0,4620,476
4,A Felis Company,83878211395,tent and awning shops,4.32,93165.80,81,1150.0,224.0,3177,584
...,...,...,...,...,...,...,...,...,...,...
3378,Vitae Orci Phasellus LLP,59614227972,"lawn and garden supply outlets, including nurs...",3.72,112488.42,93,1210.0,232.0,3128,682
3379,Vivamus Limited,22614311069,"equipment, tool, furniture, and appliance rent...",6.34,179856.93,147,1224.0,633.0,4670,614
3380,Vivamus Sit Associates,85726387922,health and beauty spas,6.28,101880.04,258,395.0,355.0,5266,868
3381,Viverra Maecenas Iaculis Associates,78522984877,"furniture, home furnishings and equipment shop...",5.57,370083.51,2132,174.0,1145.0,3915,643


In [16]:
merged_df.to_csv(r'../data/curated/merchant_detail_with_income.csv', index = False, header=True)